# Решение задачи классификации

## Оглавление

- [Постановка задачи](#item_intro)
- [Описание и загрузка данных](#item_load)
- [Первичная обработка данных](#item_prepare)
- [Разведывательный анализ данных (EDA)](#item_eda)
    - [Исследование данных таблиц `data_newbies` и `data_participated`](#item_eda_detalis)
- [Отбор и преобразование признаков](#item_fi)
    - [План создания новых признаков](#item_fi_scheme)
    - [Создание и исследование новых признаков](#item_fi_create)
    - [Выводы по новым признакам](#item_fi_conclusion)
    - [Кодирование оставшихся признаков](#item_fi_encode)
    - [Отбор лучших признаков и их нормализация](#item_fi_normalize)
- [Решение задачи классификации](#item_classifier)
- [Вывод](#item_conclusion)

## Постановка задачи <a id="item_intro"></a>

**Бизнес-задача:** определить характеристики, по которым можно выявить клиентов, \
более склонных к открытию депозита в банке, \
и за счёт этого повысить результативность маркетинговой кампании.

**Техническая задача для специалиста Data Science**: \
построить модель машинного обучения, которая на основе предложенных характеристик клиента \
будет предсказывать, воспользуется он предложением об открытии депозита или нет.

## Описание и загрузка данных <a id="item_load"></a>

Исходные данные можно скачать [здесь](https://disk.yandex.ru/d/agA97zeCmGdpuw).

Признаки в данных можно разделить на следующие группы.

**Данные о клиентах банка:**

- `age` - возраст;
- `job` - сфера занятости;
- `marital` - семейное положение;
- `education` - уровень образования;
- `default` - имеется ли просроченный кредит;
- `housing` - имеется ли кредит на жильё;
- `loan` - имеется ли кредит на личные нужды;
- `balance` -  баланс.

**Данные, связанные с последним контактом в контексте текущей маркетинговой кампании:**

- `contact` - тип контакта с клиентом;
- `month` - месяц, в котором был последний контакт;
- `day` - день, в который был последний контакт;
- `duration` - продолжительность контакта в секундах.

**Прочие признаки:**

- `campaign` - количество контактов с этим клиентом в течение текущей кампании;
- `pdays` - количество пропущенных дней с момента последней маркетинговой кампании до контакта в текущей кампании;
- `previous` - количество контактов до текущей кампании;
- `poutcome` - результат прошлой маркетинговой кампании.

И целевая переменная `deposit`, \
которая определяет, согласился ли клиент открыть депозит в банке.\
Её нужно предсказать.

Загрузим необходимые библиотеки и данные.

In [1]:
import pandas as pd

from classes.PrepareDataNewbies import columns_to_drop as columns_to_drop_from_newbies
from classes.PrepareDataParticipated import columns_to_drop as columns_to_drop_from_participated

from functions.display_content import display_field_descriptions

In [2]:
# Загрузим данные
data_source = pd.read_csv('./data/bank_fin.csv', sep=';')

# Сделаем копию таблицы
data = data_source.copy()

# Выведем первые строки
data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,"2 343,00 $",yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,"45,00 $",no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,"1 270,00 $",yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,"2 476,00 $",yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,"184,00 $",no,no,unknown,5,may,673,2,-1,0,unknown,yes


## Первичная обработка данных <a id="item_prepare"></a>

Описание первичной обработки данных для удобства чтения вынесено в файл [2_1_prepare_data.ipynb](./2_1_prepare_data.ipynb).

Исследования показывают, что данные лучше разделить на две таблицы.

- В одной оставить только данные клиентов, которые не участвовали ранее в кампаниях банка, \
назовем ее `data_newbies`.
- В другую поместить данные клиентов, которые уже участвовали в кампаниях банка,\
назовем ее `data_participated`.

В таблице `data_newbies` (с новыми клиентами) удалены поля `pdays`, `previous`, `poutcome`.\
Так как они содержат информацию об участии клиентов в прошлых кампаниях банка,\
и не информативны для новых клиентов.

Операции по обработке данных по результатам исследования вынесены методы классов:
- [classes.PrepareData](./classes/PrepareData.py) - содержит общие для обеих таблиц операции;
- [classes.PrepareDataNewbies](./classes/PrepareDataNewbies.py) - содержит операции, специфические для таблицы `data_newbies`;
- [classes.PrepareDataParticipated](./classes/PrepareDataParticipated.py) - содержит операции, специфические для таблицы `data_participated`.

Первичную обработку данных производит метод `specific_prepare_data`.

## Разведывательный анализ данных (EDA) <a id="item_eda"></a>

<h3>Исследование данных таблиц<br /> <code>data_newbies</code> и <code>data_participated</code> <a id="item_eda_detalis"></a></h3>

- Исследование данных таблицы `data_newbies` \
вынесено в файл [3_2_eda_newbies.ipynb](./3_2_eda_newbies.ipynb)

- Исследование данных таблицы `data_participated` \
вынесено в файл [3_2_eda_participated.ipynb](./3_2_eda_participated.ipynb)

Исследование показывает, что для **обеих таблиц**

*Можно выделить следующие признаки клиентов, которые более склонны открывать депозит.*

- Чем больше баланс, тем более склонны клиенты открывать депозит (признак `balance`).
- Клиенты в возрасте до 24 лет и после 60 (признак `age`).\
По сферам занятости возраст до 24 лет можно предположительно отнести к студентам,\
после 60 - к пенсионерам.
- Клиенты с высшим образованием (признак `education`).
- Клиенты, которые не брали кредита на жилье или личные нужды и у которых нет просроченного кредита \
(признаки `housing`, `loan`, `default`).
- Чем дольше длился разговор с клиентом, тем более склонен он к открытию депозита (признак `duration`).

*Малозначимые признаки*

- Зависимости от признака `campaign` (количество контактов с клиентов в рамках текущей кампании) практически не заметно.

*Также наблюдается взаимосвязь признаков*

- `age` (возраст) и `marital` (семейное положение).\
Причем в двух таблицах характер их распределения немного разный.
- `job` (сфера занятости) и `education` (уровень образования).\
Среди менеджеров больше клиентов с высшим образованием, \
а среди "синих воротничков" - с начальным.

**Для таблицы `data_newbies`**

*Можно выделить следующие признаки клиентов, которые более склонны открывать депозит.*

- Одинокие и разведенные клиенты (признак `marital`).
- Студенты, пенсионеры, безработные (признак `job`).
- Клиенты, которым в последний раз звонили в апреле, марте, сентябре, октябре и декабре (признак `month`).
- Клиенты, способ контакта с которыми известен (звонок на сотовый или стационарный телефон) (признак `contact`).

*Наиболее значимые признаки*

- Наиболее значимым признаком является `duration` (длительность последнего общения с клиентом).\
Коэффициент корреляции с `deposit` равен `0.57`.

*Наблюдается взаимосвязь следующих признаков*

-  `month` (месяц последнего контакта) и `housing` (наличие кредита на жилье).\
Подоавляющее большинство клиентов, с которыми связывались с мае, взяли кредит на жилье.\
Возможно, в мае была какая-то акция, связанная с выгодным кредитом на жилье.

- `month` (месяц последнего контакта) и `contact` (тип контакта с клиентом)\
Наибольшее количество контактов с типом `unknown` (неизвестный) наблюдается в мае и июне.\
Возможно, это связано с особенностями работы банка в эти месяцы.

**Для таблицы `data_participated`**

*Можно выделить следующие признаки клиентов, которые более склонны открывать депозит.*

- Студенты, пенсионеры, безработные, а также менеджеры (признак `job`).
- Клиенты, которым последний раз звонили \
в марте, июне, июле, августе, сентябре, октябре и декабре (признак `month`).
- Клиенты, которым звонили на сотовый (признак `contact`).
- Клиенты, у для которых результат прошлой кампании банка был успешным (признак `poutcome`).
- Клиенты, с которыми времени общения с момента прошлой кампании прошло меньше (признак `pdays`).

*Малозначимые признаки*

- Доля не открывших депозит среди женатых, одиноких и разведенных клиентов примерно одинакова.\
Для данной таблицы этот признак (`marital`) малозначим.
- Зависимости от признака `previous` (количество контактов до текущей кампании) практически не заметно.

*Наиболее значимые признаки*

- Три наиболее значимых признака  
    - `poutcome` (успех предыдущей кампании)
    - `housing` (наличие кредита на жилье)
    - `duration` (длительность последнего общения с клиентом)
    
Характер их влияния на открытие депозита описан выше.

*Наблюдается взаимосвязь признаков*

- `month` (месяц последнего контакта) и `housing` (наличие кредита на жилье).\
Подавляющее большинство клиентов, с которыми связывались с мае и апреле, взяли кредит на жилье.\
Возможно, в мае-апреле была какая-то акция, связанная с выгодным кредитом на жилье.

В рамках исследования изменения в таблицы не вносились.

## Отбор и преобразование признаков <a id="item_fi"></a>

### План создания новых признаков <a id="item_fi_scheme"></a>

На основе исследования данных есть предположение, что имеет смысл создать следующие новые признаки.

**Для обеих таблиц:**

- На основе признака `job` сделать признак `has_income`, в который выделить 2 категории клиентов:
    - без дохода (студенты, пенсионеры, безработные);
    - с доходом (все остальные).

- На основе признака `job` создать порядковый признак `job_loyal`.\
И в каждой группе собрать сферы занятости, где процентное соотношение открывших и не открывших депозит примерно одинаково.

- На основе признаков `housing`, `loan`, `default` сделать признак `has_credit`, \
который говорит о том, есть ли у клиента кредит.

- На основе признака `balance` создать признак `is_debtor`, \
который будет содержать информацию о том, отрицательный у клиента баланс или нет.

- Разделить признак `age` по группам в 10 лет (создать новый признак `age_scale`). \
И посмотреть, даст ли он бОльшую корреляцию с признаком `deposit`.

- На основе признака `age` создать признак `is_working_age` (является ли возраст рабочим),\
который будет принимать `1`, если возраст клиента от `25` до `60` лет.\
И `0` в противном случае.

- Разделить признак `duration` по группам в 10 минут (создать новый признак `duration_scale`). \
И посмотреть, даст ли он бОльшую корреляцию с признаком `deposit`.

- На основе признака `month` сделать признак `season` с временем года, когда был последний контакт.

- Объединить признаки `month` и `day` в признак `contact_date` с календарными датами и посмотреть, \
есть ли периоды, в которые клиенты более склонны открывать депозит.

- При исследовании распределения признака `contact_date` можно заметить, что в определенные промежутки
клиентам звонили явно чаще, а в другие явно реже.\
Также в одни промежутки клиенты явно больше брали кредит на жилье, в другие открывали депозит.\
Можно предположить, что в течение года (к сожалению, не известно, имеем ли мы дело с данными за один год или за несколько) в разные даты есть прозвоны разных типов:\
    - по кредиту на жилье
    - по открытию депозита
    - общий прозвон (и кредит на жилье и депозит)
    - отсутствие прозвона (спокойное время)\    
Назовем этот признак `ringing_type`.\
Если данные представлены за один год, то в другой год могут быть другие типы прозвонов, и этот признак будет бесполезен, нужно будет искать другие закономерности.

### Создание и исследование новых признаков <a id="item_fi_create"></a>

Код создания новых признаков вынесен в метод `add_new_specific_features` \
классов [PrepareDataNewbies](./classes/PrepareDataNewbies.py) и [PrepareDataParticipated](./classes/PrepareDataParticipated.py).

Список новых признаков находится в переменной `new_features` в общем классе [PrepareData](./classes/PrepareData.py).

Создание новых признаков и исследование их корреляции с целевой переменной `deposit` вынесено в файлы:

- Для таблицы `data_newbies` \
в файл [4_2_feature_inginiring_newbies.ipynb](./4_2_feature_inginiring_newbies.ipynb)
- Для таблицы `data_participated` \
в файл [4_2_feature_inginiring_participated.ipynb](./4_2_feature_inginiring_participated.ipynb)

### Выводы по новым признакам <a id="item_fi_conclusion"></a>

По результатам исследования можно сделать вывод, что эффективными для обеих таблиц оказались признаки:

- `has_credit` (есть ли кредит у клиента). \
Он имеет корреляцию с `deposit` больше, чем исходные `housing`, `loan`, `default`.\
По распределению видно, что клиенты, у которых есть кредит, менее склонны открывать депозит.

- `is_working_age` (является ли возраст клиента рабочим).\
Имеет бОльшую корреляцию с `deposit`, чем исходный `age`.\
По распределению видно, что клиенты в нерабочем возрасте более склонны открывать депозит.

- `ringing_type` (тип прозвона).\
Также имеет бОльшую корреляцию с `deposit`, чем исходный `contact_date`.\
При типах прозвона `weak` и `deposit` клиенты открывают депозит чаще.

- `has_income` (имеется ли доход у клиента).\
Имеет бОльшую корреляцию с `deposit`, чем исходный `job`.\
По распределению видно, что клиенты, у которых нет дохода, открывают депозит чаще.

- `job_loyal` (уровень лояльности к предложению открыть депозит в зависимости от сферы занятости).\
Также имеет корреляцию с `deposit` больше, чем исходный `job`.\
И в общем включает в себя информацию из признака `has_income`.

Перед кодированием признаков из обеих таблиц удалим менее значимые признаки,\
которые коррелируют с существующими и по сути повторяют информацию в них.\
Список их для каждой таблицы определен эксперементально.

Из таблицы `data_newbies` удалим признаки:

In [3]:
display_field_descriptions(columns_to_drop_from_newbies)

- `duration_scale` - шкала длительности последнего контакта

- `age_scale` - шкала возраста

- `housing` - имеется ли кредит на жильё

- `loan` - имеется ли кредит на личные нужды

- `default` - имеется ли просроченный кредит

- `season` - время года последнего контакта с клиентом

- `job` - сфера занятости

- `month` - месяц, в котором был последний контакт

- `day` - день, в который был последний контакт

- `age` - возраст

Из таблицы `data_participated` удалим признаки:

In [4]:
display_field_descriptions(columns_to_drop_from_participated)

- `age` - возраст

- `housing` - имеется ли кредит на жильё

- `loan` - имеется ли кредит на личные нужды

- `default` - имеется ли просроченный кредит

- `duration_scale` - шкала длительности последнего контакта

- `season` - время года последнего контакта с клиентом

- `month` - месяц, в котором был последний контакт

- `has_income` - имеется ли доход у клиента

- `job` - сфера занятости

- `contact_date` - дата, когда был последний контакт

### Кодирование оставшихся признаков <a id="item_fi_encode"></a>

В обеих таблицах все признаки с типом `object` подходят для однократного кодирования.

Признак `education` сделаем порядковым, а признак `marital` бинарным.

Столбцы `contact_date` и `day` являются по сути категориальными.\
Поскольку они имеют больше 15 уникальных значений, закодируем их с помощью бинарного кодирования.

Итоговый код кодирования признаков реализован в методе `encoded_specific_columns` \
классов [PrepareDataNewbies](./classes/PrepareDataNewbies.py) и [PrepareDataParticipated](./classes/PrepareDataParticipated.py).

Кодирование признаков и их дальнейшее исследование находится в тех же файлах.

- Для таблицы `data_newbies` 
в [4_2_feature_inginiring_newbies.ipynb](./4_2_feature_inginiring_newbies.ipynb)

- Для таблицы `data_participated` 
в [4_2_feature_inginiring_participated.ipynb](./4_2_feature_inginiring_participated.ipynb)

### Отбор лучших признаков и их нормализация <a id="item_fi_normalize"></a>

Отбор лучших признаков реализован с помощью `SelectKBest`.

Количество оставляемых признаков определено эксперементально и вынесено в поле `best_features_count` \
классов [PrepareDataNewbies](./classes/PrepareDataNewbies.py) и [PrepareDataParticipated](./classes/PrepareDataParticipated.py).

Нормализация реализована  с помощью `MinMaxScaler`

Итоговый код для подготовки данных для модели реализован в методе `get_specific_data_for_model` упомянутых выше классов.

## Решение задачи классификации <a id="item_classifier"></a>

> Для оценки качества моделей выберем метрику $F_1$, так как выборка для `data_participated` несбалансирована.

Для обеих таблиц (`data_newbies` и `data_participated`) реализована классификацию с помощью моделей:
- Логистическая регрессия
- Дерево решений
- Случайный лес
- Градиентный бустинг
- И стекинг на основе полученных моделей

Гиперпараметры моделей подобраны 2-мя способами:
- визуально с помощью графика;
- с помощью библиотеки `optuna`.

Сделано сравнение, какие модели дают лучший результат.

Для таблицы `data_newbies` решение задачи классификации вынесено  в файл [5_2_classification_newbies.ipynb](./5_2_classification_newbies.ipynb).\
А для таблицы `data_participated` в файл [5_2_classification_participated.ipynb](./5_2_classification_participated.ipynb).

## Вывод <a id="item_conclusion"></a>

**Для таблицы `data_newbies`** лучший результат показала модель `StackingClassifier`.\
Значение метрики $F_1$ на тестовой выборке равно `0.8392`.

**Для таблицы `data_participated`** лучший результат показала модель `RandomForestClassifier`.\
Значение метрики $F_1$ на тестовой выборке равно `0.8703`.

**Для обеих таблиц** можно заметить, что поиск оптимального значения гиперпараметров\
с помощью графика дает сравнимый, а иногда и более лучший результат, \
чем получение параметров с помощью `optuna`.

Чтобы получить больше пользы от `optuna`, нужно поэтапно корректировать область поиска параметров.